In [35]:
# Alexandre Brilhante

import sqlite3
import pandas as pd
import numpy as np
import datetime

db = 'database.sqlite'
connect = sqlite3.connect(db)
query = "SELECT name FROM sqlite_master WHERE type = 'table';"
pd.read_sql(query, connect)

,name
0,sqlite_sequence
1,Player_Attributes
2,Player
3,Match
4,League
5,Country
6,Team
7,Team_Attributes


In [36]:
# Initialize

query = "SELECT * FROM player;"
players_df = pd.read_sql(query, connect)

query = "SELECT * FROM player_attributes"
player_stats_df = pd.read_sql(query, connect)

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

In [37]:
# Merge the player and player_attribute data
df = players_df.merge(player_stats_df, how='inner', on='player_api_id')

df['date'] = pd.to_datetime(df['date'])

# Unnecessary columns
df.drop(['id_x', 'id_y', 'player_fifa_api_id_x', 'player_fifa_api_id_y'], 1, inplace=True)
ratings_df = df[['player_api_id', 'player_name', 'date', 'overall_rating', 'potential']]

# Drop players without any rating
ratings_df = ratings_df.drop(ratings_df[ratings_df['overall_rating'].isnull()].index)

# Sorting by rating rather than age
ratings_df.sort_values(['player_name', 'player_api_id', 'overall_rating'],
                       ascending=[True, True, False], inplace=True)

# Change the date to just the year
ratings_df['date'] = ratings_df['date'].apply(lambda x: x.year)

In [44]:
# Grouping the players by the year
group = ratings_df.groupby('date')

# Dropping the duplicate player entries per year
ratings_df_unique = group.apply(lambda x: x.drop_duplicates(subset = 'player_api_id', keep = 'first'))

# Grouping the df again into another df group object
group = ratings_df_unique.groupby('date')

Ranking players based on overall ratings

In [50]:
data = group.apply(lambda x: x.sort_values(by='overall_rating', ascending=[False]).sort_values(
    by=['overall_rating'], ascending=[False]).reset_index(drop=True).head(15))

In [51]:
""" Best team possible for each year. """

data

player_api_id             player_name  date  overall_rating  \
date                                                                   
2007 0           30829            Wayne Rooney  2007            93.0   
     1           30717        Gianluigi Buffon  2007            93.0   
     2           39989          Gregory Coupet  2007            92.0   
     3           30893       Cristiano Ronaldo  2007            91.0   
     4           30743              Ronaldinho  2007            91.0   
     5           30626           Thierry Henry  2007            91.0   
     6           30627              John Terry  2007            91.0   
     7           30723        Alessandro Nesta  2007            91.0   
     8           34520         Fabio Cannavaro  2007            91.0   
     9           38843              Ze Roberto  2007            90.0   
     10          41044                    Kaka  2007            90.0   
     11          30714         Francesco Totti  2007            90.0   
     12          30728         David Trezeguet  2007            90.0   
     13          30631           Frank Lampard  2007            90.0   
     14          30709               Luca Toni  2007            89.0   
2008 0           30657           Iker Casillas  2008            91.0   
     1           30893       Cristiano Ronaldo  2008            91.0   
     2           30717        Gianluigi Buffon  2008            90.0   
     3           30981            Lionel Messi  2008            90.0   
     4           41044                    Kaka  2008            90.0   
     5           35724      Zlatan Ibrahimovic  2008            89.0   
     6           30859               Petr Cech  2008            88.0   
     7           30613           Cesc Fabregas  2008            88.0   
     8           30924           Franck Ribery  2008            88.0   
     9           30853         Fernando Torres  2008            88.0   
     10          30661            Carles Puyol  2008            88.0   
     11          30618          Steven Gerrard  2008            88.0   
     12          30723        Alessandro Nesta  2008            88.0   
     13          30627              John Terry  2008            88.0   
     14          39854          Xavi Hernandez  2008            87.0   
2009 0           30657           Iker Casillas  2009            91.0   
     1           30717        Gianluigi Buffon  2009            90.0   
     2           30981            Lionel Messi  2009            90.0   
     3           30893       Cristiano Ronaldo  2009            90.0   
     4           30989             Julio Cesar  2009            89.0   
     5           41044                    Kaka  2009            89.0   
     6           35724      Zlatan Ibrahimovic  2009            89.0   
     7           30859               Petr Cech  2009            88.0   
     8           30618          Steven Gerrard  2009            88.0   
     9           30661            Carles Puyol  2009            88.0   
     10          30865           Nemanja Vidic  2009            88.0   
     11          30924           Franck Ribery  2009            88.0   
     12          30853         Fernando Torres  2009            88.0   
     13          30613           Cesc Fabregas  2009            87.0   
     14          31235                   Diego  2009            87.0   
2010 0           30981            Lionel Messi  2010            90.0   
     1           30657           Iker Casillas  2010            90.0   
     2           30989             Julio Cesar  2010            89.0   
     3           30893       Cristiano Ronaldo  2010            89.0   
     4           30829            Wayne Rooney  2010            89.0   
     5           30661            Carles Puyol  2010            88.0   
     6           30822           Didier Drogba  2010            88.0   
     7           30717        Gianluigi Buffon  2010            88.0   
     8           30613           Cesc Fabregas  2

In [49]:
data.groupby('date').sum().sort_values(['overall_rating'], ascending=[False])

,player_api_id,overall_rating,potential
date,,,
2007,492336,1363.0,1378.0
2012,481456,1344.0,1365.0
2011,487372,1343.0,1363.0
2014,607749,1342.0,1351.0
2013,531335,1340.0,1354.0
2015,613385,1338.0,1345.0
2008,485748,1332.0,1374.0
2009,477633,1330.0,1377.0
2010,466758,1326.0,1371.0


In [48]:
""" The best team is 2007. """

' The best team is 2007. '